# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [13]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    
    source_id_text = []
    target_id_text = []
    
    for line in source_text.split('\n'):
        sentence = []
        for word in line.split():
            sentence.append(source_vocab_to_int[word])
        source_id_text.append(sentence)
            
    for line in target_text.split('\n'):
        sentence = []
        for word in line.split():
            sentence.append(target_vocab_to_int[word])
        target_id_text.append(sentence + [source_vocab_to_int['<EOS>']])
   
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [14]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


/Users/danilo.herrera/anaconda/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [3]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    input = tf.placeholder(tf.int32, shape=[None, None], name="input")
    targets = tf.placeholder(tf.int32, shape=[None, None], name="targets")
    learning_rate = tf.placeholder(tf.float32, name="learning_rate")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    target_sequence_length = tf.placeholder(tf.int32, shape=[None], name="target_sequence_length")
    max_target_len = tf.reduce_max(target_sequence_length, name="max_target_len")
    source_sequence_length = tf.placeholder(tf.int32, shape=[None], name="source_sequence_length")
    return input, targets, learning_rate, keep_prob, target_sequence_length, max_target_len, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [4]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    go_id = tf.constant(target_vocab_to_int["<GO>"], shape=[batch_size, 1])
    decoded_input = tf.concat([go_id, target_data[:, :-1]], 1)
    return decoded_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [5]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    lstm_cells = []
    for _ in range(num_layers):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        lstm_cells.append(lstm)
    multi_rnn_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells)
    embedding = tf.contrib.layers.embed_sequence(rnn_inputs, vocab_size=source_vocab_size, embed_dim=encoding_embedding_size)
    outputs, state = tf.nn.dynamic_rnn(multi_rnn_cell, embedding, source_sequence_length, dtype=tf.float32)
    return outputs, state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [6]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    training_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper, encoder_state, output_layer)
    final_outputs, _ = tf.contrib.seq2seq.dynamic_decode(basic_decoder,maximum_iterations=max_summary_length)
    return final_outputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [7]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    tiled_ids = tf.tile([start_of_sequence_id], [batch_size])
    embedding_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, tiled_ids, end_of_sequence_id)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, embedding_helper, encoder_state, output_layer)
    final_outputs, _ = tf.contrib.seq2seq.dynamic_decode(basic_decoder,maximum_iterations=max_target_sequence_length)
    return final_outputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [8]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    lstm_cells = []
    for _ in range(num_layers):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        lstm_cells.append(lstm)
    multi_rnn_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells)
    decoding_embedding = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    embedded_input = tf.nn.embedding_lookup(decoding_embedding, dec_input)
    dense_layer = Dense(target_vocab_size, kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))
    with tf.variable_scope("decoding") as scope:
        trained_output = decoding_layer_train(encoder_state, multi_rnn_cell, embedded_input,
                                                 target_sequence_length, max_target_sequence_length,
                                                 dense_layer, keep_prob)
        scope.reuse_variables()
        inferred_output = decoding_layer_infer(encoder_state, multi_rnn_cell, decoding_embedding,
                                                  target_vocab_to_int["<GO>"], target_vocab_to_int["<EOS>"],
                                                  max_target_sequence_length, target_vocab_size,
                                                  dense_layer, batch_size, keep_prob)
    return trained_output, inferred_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [9]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    outputs, state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, source_sequence_length,
                                                   source_vocab_size, enc_embedding_size)
    decoded_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    training_output, inferred_output = decoding_layer(decoded_input, state, target_sequence_length, 
                                                        max_target_sentence_length, rnn_size,num_layers, 
                                                        target_vocab_to_int, target_vocab_size,batch_size, 
                                                        keep_prob, dec_embedding_size)
    return training_output, inferred_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [14]:
# Number of Epochs
epochs = 5
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 5
# Embedding Size
encoding_embedding_size = 256
decoding_embedding_size = 256
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.7
display_step = 1

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/538 - Train Accuracy: 0.2316, Validation Accuracy: 0.3161, Loss: 5.7190
Epoch   0 Batch    2/538 - Train Accuracy: 0.2518, Validation Accuracy: 0.3161, Loss: 5.0885
Epoch   0 Batch    3/538 - Train Accuracy: 0.2295, Validation Accuracy: 0.3166, Loss: 4.6516
Epoch   0 Batch    4/538 - Train Accuracy: 0.2703, Validation Accuracy: 0.3450, Loss: 4.6652
Epoch   0 Batch    5/538 - Train Accuracy: 0.2978, Validation Accuracy: 0.3464, Loss: 4.2084
Epoch   0 Batch    6/538 - Train Accuracy: 0.3025, Validation Accuracy: 0.3466, Loss: 4.0446
Epoch   0 Batch    7/538 - Train Accuracy: 0.2848, Validation Accuracy: 0.3509, Loss: 4.0830
Epoch   0 Batch    8/538 - Train Accuracy: 0.2799, Validation Accuracy: 0.3471, Loss: 3.9153
Epoch   0 Batch    9/538 - Train Accuracy: 0.2977, Validation Accuracy: 0.3633, Loss: 3.7584
Epoch   0 Batch   10/538 - Train Accuracy: 0.2922, Validation Accuracy: 0.3770, Loss: 3.7813
Epoch   0 Batch   11/538 - Train Accuracy: 0.3082, Validation Accuracy

Epoch   0 Batch   90/538 - Train Accuracy: 0.4723, Validation Accuracy: 0.5020, Loss: 2.2394
Epoch   0 Batch   91/538 - Train Accuracy: 0.4598, Validation Accuracy: 0.5154, Loss: 2.3136
Epoch   0 Batch   92/538 - Train Accuracy: 0.4691, Validation Accuracy: 0.5110, Loss: 2.2455
Epoch   0 Batch   93/538 - Train Accuracy: 0.4623, Validation Accuracy: 0.5174, Loss: 2.2877
Epoch   0 Batch   94/538 - Train Accuracy: 0.4387, Validation Accuracy: 0.4968, Loss: 2.3001
Epoch   0 Batch   95/538 - Train Accuracy: 0.5138, Validation Accuracy: 0.5067, Loss: 2.0836
Epoch   0 Batch   96/538 - Train Accuracy: 0.5000, Validation Accuracy: 0.5186, Loss: 2.1030
Epoch   0 Batch   97/538 - Train Accuracy: 0.4656, Validation Accuracy: 0.5199, Loss: 2.2525
Epoch   0 Batch   98/538 - Train Accuracy: 0.4978, Validation Accuracy: 0.5076, Loss: 2.0713
Epoch   0 Batch   99/538 - Train Accuracy: 0.4596, Validation Accuracy: 0.5172, Loss: 2.2741
Epoch   0 Batch  100/538 - Train Accuracy: 0.4861, Validation Accuracy

Epoch   0 Batch  179/538 - Train Accuracy: 0.5004, Validation Accuracy: 0.5297, Loss: 1.7363
Epoch   0 Batch  180/538 - Train Accuracy: 0.5242, Validation Accuracy: 0.5440, Loss: 1.6874
Epoch   0 Batch  181/538 - Train Accuracy: 0.4813, Validation Accuracy: 0.5483, Loss: 1.7822
Epoch   0 Batch  182/538 - Train Accuracy: 0.4719, Validation Accuracy: 0.5369, Loss: 1.7395
Epoch   0 Batch  183/538 - Train Accuracy: 0.5087, Validation Accuracy: 0.5341, Loss: 1.6180
Epoch   0 Batch  184/538 - Train Accuracy: 0.5158, Validation Accuracy: 0.5284, Loss: 1.6205
Epoch   0 Batch  185/538 - Train Accuracy: 0.4844, Validation Accuracy: 0.5337, Loss: 1.7370
Epoch   0 Batch  186/538 - Train Accuracy: 0.5162, Validation Accuracy: 0.5426, Loss: 1.6597
Epoch   0 Batch  187/538 - Train Accuracy: 0.5410, Validation Accuracy: 0.5487, Loss: 1.5726
Epoch   0 Batch  188/538 - Train Accuracy: 0.4602, Validation Accuracy: 0.5204, Loss: 1.6960
Epoch   0 Batch  189/538 - Train Accuracy: 0.4979, Validation Accuracy

Epoch   0 Batch  268/538 - Train Accuracy: 0.5234, Validation Accuracy: 0.5265, Loss: 1.2907
Epoch   0 Batch  269/538 - Train Accuracy: 0.5078, Validation Accuracy: 0.5531, Loss: 1.4019
Epoch   0 Batch  270/538 - Train Accuracy: 0.5053, Validation Accuracy: 0.5522, Loss: 1.3503
Epoch   0 Batch  271/538 - Train Accuracy: 0.5215, Validation Accuracy: 0.5581, Loss: 1.3670
Epoch   0 Batch  272/538 - Train Accuracy: 0.4898, Validation Accuracy: 0.5510, Loss: 1.4226
Epoch   0 Batch  273/538 - Train Accuracy: 0.5279, Validation Accuracy: 0.5614, Loss: 1.3708
Epoch   0 Batch  274/538 - Train Accuracy: 0.4869, Validation Accuracy: 0.5582, Loss: 1.3960
Epoch   0 Batch  275/538 - Train Accuracy: 0.5162, Validation Accuracy: 0.5499, Loss: 1.3949
Epoch   0 Batch  276/538 - Train Accuracy: 0.5178, Validation Accuracy: 0.5499, Loss: 1.3608
Epoch   0 Batch  277/538 - Train Accuracy: 0.5008, Validation Accuracy: 0.5510, Loss: 1.3305
Epoch   0 Batch  278/538 - Train Accuracy: 0.4881, Validation Accuracy

Epoch   0 Batch  357/538 - Train Accuracy: 0.5049, Validation Accuracy: 0.5225, Loss: 1.2114
Epoch   0 Batch  358/538 - Train Accuracy: 0.4805, Validation Accuracy: 0.5149, Loss: 1.1767
Epoch   0 Batch  359/538 - Train Accuracy: 0.5233, Validation Accuracy: 0.5121, Loss: 1.0912
Epoch   0 Batch  360/538 - Train Accuracy: 0.4752, Validation Accuracy: 0.5183, Loss: 1.1860
Epoch   0 Batch  361/538 - Train Accuracy: 0.5281, Validation Accuracy: 0.5328, Loss: 1.1048
Epoch   0 Batch  362/538 - Train Accuracy: 0.5115, Validation Accuracy: 0.5279, Loss: 1.1157
Epoch   0 Batch  363/538 - Train Accuracy: 0.5195, Validation Accuracy: 0.5195, Loss: 1.1286
Epoch   0 Batch  364/538 - Train Accuracy: 0.4773, Validation Accuracy: 0.5444, Loss: 1.1712
Epoch   0 Batch  365/538 - Train Accuracy: 0.5188, Validation Accuracy: 0.5227, Loss: 1.2317
Epoch   0 Batch  366/538 - Train Accuracy: 0.4707, Validation Accuracy: 0.5128, Loss: 1.1953
Epoch   0 Batch  367/538 - Train Accuracy: 0.4863, Validation Accuracy

...

Epoch   4 Batch  171/538 - Train Accuracy: 0.8229, Validation Accuracy: 0.7816, Loss: 0.2336
Epoch   4 Batch  172/538 - Train Accuracy: 0.8175, Validation Accuracy: 0.7892, Loss: 0.2159
Epoch   4 Batch  173/538 - Train Accuracy: 0.8224, Validation Accuracy: 0.7983, Loss: 0.2118
Epoch   4 Batch  174/538 - Train Accuracy: 0.7887, Validation Accuracy: 0.7962, Loss: 0.2249
Epoch   4 Batch  175/538 - Train Accuracy: 0.7902, Validation Accuracy: 0.7967, Loss: 0.2328
Epoch   4 Batch  176/538 - Train Accuracy: 0.8100, Validation Accuracy: 0.7978, Loss: 0.2539
Epoch   4 Batch  177/538 - Train Accuracy: 0.8186, Validation Accuracy: 0.7994, Loss: 0.2266
Epoch   4 Batch  178/538 - Train Accuracy: 0.8092, Validation Accuracy: 0.7937, Loss: 0.2278
Epoch   4 Batch  179/538 - Train Accuracy: 0.7973, Validation Accuracy: 0.8018, Loss: 0.2228
Epoch   4 Batch  180/538 - Train Accuracy: 0.8218, Validation Accuracy: 0.8072, Loss: 0.2183
Epoch   4 Batch  181/538 - Train Accuracy: 0.7758, Validation Accuracy

Epoch   4 Batch  260/538 - Train Accuracy: 0.7950, Validation Accuracy: 0.8232, Loss: 0.2081
Epoch   4 Batch  261/538 - Train Accuracy: 0.8314, Validation Accuracy: 0.8224, Loss: 0.2184
Epoch   4 Batch  262/538 - Train Accuracy: 0.8230, Validation Accuracy: 0.8272, Loss: 0.2130
Epoch   4 Batch  263/538 - Train Accuracy: 0.8164, Validation Accuracy: 0.8176, Loss: 0.2093
Epoch   4 Batch  264/538 - Train Accuracy: 0.8119, Validation Accuracy: 0.8205, Loss: 0.2174
Epoch   4 Batch  265/538 - Train Accuracy: 0.7990, Validation Accuracy: 0.8171, Loss: 0.2222
Epoch   4 Batch  266/538 - Train Accuracy: 0.8363, Validation Accuracy: 0.8146, Loss: 0.2169
Epoch   4 Batch  267/538 - Train Accuracy: 0.8100, Validation Accuracy: 0.8166, Loss: 0.2220
Epoch   4 Batch  268/538 - Train Accuracy: 0.8491, Validation Accuracy: 0.8123, Loss: 0.1988
Epoch   4 Batch  269/538 - Train Accuracy: 0.8049, Validation Accuracy: 0.8091, Loss: 0.2154
Epoch   4 Batch  270/538 - Train Accuracy: 0.8092, Validation Accuracy

Epoch   4 Batch  349/538 - Train Accuracy: 0.8494, Validation Accuracy: 0.8285, Loss: 0.1766
Epoch   4 Batch  350/538 - Train Accuracy: 0.8499, Validation Accuracy: 0.8347, Loss: 0.2089
Epoch   4 Batch  351/538 - Train Accuracy: 0.8398, Validation Accuracy: 0.8453, Loss: 0.2146
Epoch   4 Batch  352/538 - Train Accuracy: 0.8322, Validation Accuracy: 0.8464, Loss: 0.2121
Epoch   4 Batch  353/538 - Train Accuracy: 0.8400, Validation Accuracy: 0.8411, Loss: 0.2009
Epoch   4 Batch  354/538 - Train Accuracy: 0.8363, Validation Accuracy: 0.8453, Loss: 0.2076
Epoch   4 Batch  355/538 - Train Accuracy: 0.8457, Validation Accuracy: 0.8432, Loss: 0.2032
Epoch   4 Batch  356/538 - Train Accuracy: 0.8359, Validation Accuracy: 0.8354, Loss: 0.1726
Epoch   4 Batch  357/538 - Train Accuracy: 0.8342, Validation Accuracy: 0.8237, Loss: 0.1949
Epoch   4 Batch  358/538 - Train Accuracy: 0.8441, Validation Accuracy: 0.8201, Loss: 0.1885
Epoch   4 Batch  359/538 - Train Accuracy: 0.8108, Validation Accuracy

Epoch   4 Batch  438/538 - Train Accuracy: 0.8352, Validation Accuracy: 0.8295, Loss: 0.1995
Epoch   4 Batch  439/538 - Train Accuracy: 0.8654, Validation Accuracy: 0.8244, Loss: 0.2051
Epoch   4 Batch  440/538 - Train Accuracy: 0.8277, Validation Accuracy: 0.8356, Loss: 0.2313
Epoch   4 Batch  441/538 - Train Accuracy: 0.8014, Validation Accuracy: 0.8272, Loss: 0.2126
Epoch   4 Batch  442/538 - Train Accuracy: 0.8359, Validation Accuracy: 0.8365, Loss: 0.1819
Epoch   4 Batch  443/538 - Train Accuracy: 0.8192, Validation Accuracy: 0.8342, Loss: 0.2040
Epoch   4 Batch  444/538 - Train Accuracy: 0.8717, Validation Accuracy: 0.8352, Loss: 0.1923
Epoch   4 Batch  445/538 - Train Accuracy: 0.8607, Validation Accuracy: 0.8368, Loss: 0.1951
Epoch   4 Batch  446/538 - Train Accuracy: 0.8610, Validation Accuracy: 0.8425, Loss: 0.1788
Epoch   4 Batch  447/538 - Train Accuracy: 0.8313, Validation Accuracy: 0.8434, Loss: 0.1997
Epoch   4 Batch  448/538 - Train Accuracy: 0.8501, Validation Accuracy

Epoch   4 Batch  527/538 - Train Accuracy: 0.8676, Validation Accuracy: 0.8645, Loss: 0.1782
Epoch   4 Batch  528/538 - Train Accuracy: 0.8592, Validation Accuracy: 0.8619, Loss: 0.1954
Epoch   4 Batch  529/538 - Train Accuracy: 0.8350, Validation Accuracy: 0.8468, Loss: 0.1789
Epoch   4 Batch  530/538 - Train Accuracy: 0.8266, Validation Accuracy: 0.8436, Loss: 0.1963
Epoch   4 Batch  531/538 - Train Accuracy: 0.8734, Validation Accuracy: 0.8436, Loss: 0.1642
Epoch   4 Batch  532/538 - Train Accuracy: 0.8619, Validation Accuracy: 0.8469, Loss: 0.1767
Epoch   4 Batch  533/538 - Train Accuracy: 0.8482, Validation Accuracy: 0.8501, Loss: 0.1817
Epoch   4 Batch  534/538 - Train Accuracy: 0.8529, Validation Accuracy: 0.8475, Loss: 0.1627
Epoch   4 Batch  535/538 - Train Accuracy: 0.8570, Validation Accuracy: 0.8562, Loss: 0.1612
Epoch   4 Batch  536/538 - Train Accuracy: 0.8683, Validation Accuracy: 0.8610, Loss: 0.1888
Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [21]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    sequence = []
    for word in sentence.lower().split():
        sequence.append(vocab_to_int.get(word, vocab_to_int["<UNK>"]))
    return sequence


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [22]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [152, 60, 75, 85, 198, 177, 142]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [268, 132, 218, 213, 104, 38, 330, 1]
  French Words: il a vu la voiture blanche . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.